# Consultas MONGODB

Nicolás Fernández y Néstor Villa

In [2]:
# Importamos las clases y el método init_app
from models import Cliente, Producto, Compra, Proveedor, init_app
from bson import ObjectId
from datetime import datetime

# Iniciamos la conexión a la base de datos
init_app()

In [5]:
#1. Listado de todas las compras de un cliente
pipeline = [
    {
        "$group": {
            "_id": "$cliente.nombre",
            "compras": { "$push": { "$toString": "$_id" } }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "compras": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
compras_cliente = list(Compra.aggregate(pipeline, raw=True))

compras_cliente

[{'compras': ['671e87d26018dfcfed045207', '671e87d36018dfcfed04520c'],
  'nombre_cliente': 'Ana Sánchez'},
 {'compras': ['671e87d26018dfcfed0451fb',
   '671e87d26018dfcfed045201',
   '671e87d26018dfcfed045208'],
  'nombre_cliente': 'Luis Martínez'},
 {'compras': ['671e87d26018dfcfed0451f6',
   '671e87d26018dfcfed0451f8',
   '671e87d26018dfcfed0451f9',
   '671e87d26018dfcfed0451fa',
   '671e87d26018dfcfed0451fd',
   '671e87d26018dfcfed0451fe',
   '671e87d26018dfcfed045205',
   '671e87d26018dfcfed045206',
   '671e87d36018dfcfed04520d'],
  'nombre_cliente': 'María Fernández'},
 {'compras': ['671e87d26018dfcfed0451f7',
   '671e87d26018dfcfed0451ff',
   '671e87d26018dfcfed045200',
   '671e87d36018dfcfed045209',
   '671e87d36018dfcfed04520f',
   '671e87d36018dfcfed045210',
   '671e87d36018dfcfed045211',
   '671e87d36018dfcfed045212',
   '671e87d36018dfcfed045213'],
  'nombre_cliente': 'Beatriz Gómez'},
 {'compras': ['671e87d26018dfcfed0451fc',
   '671e87d26018dfcfed045202',
   '671e87d26018d

In [6]:
# 2. Listado de todos los proveedores para un producto
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$group": {
            "_id": "$nombre",
            "proveedores": { "$push": "$proveedores.nombre" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$_id",
            "proveedores": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
proveedores_producto = list(Producto.aggregate(pipeline, raw=True))

proveedores_producto

[{'proveedores': ['Proveedor Este'],
  'nombre_producto': 'Calcetines de algodón'},
 {'proveedores': ['Modas Paqui'],
  'nombre_producto': 'Camiseta de manga corta roja'},
 {'proveedores': ['Proveedor Este'], 'nombre_producto': 'Sombrero de paja'},
 {'proveedores': ['Proveedor Central'],
  'nombre_producto': 'Guantes de invierno'},
 {'proveedores': ['Proveedor Este'], 'nombre_producto': 'Jersey de lana'},
 {'proveedores': ['Logística Sur'], 'nombre_producto': 'Traje de baño'},
 {'proveedores': ['Modas Paqui'],
  'nombre_producto': 'Polo de manga corta verde'},
 {'proveedores': ['Modas Paqui'], 'nombre_producto': 'Cinturón de cuero'},
 {'proveedores': ['Modas Paqui'],
  'nombre_producto': 'Camiseta de manga corta blanca'},
 {'proveedores': ['Distribuciones Norte'], 'nombre_producto': 'Falda plisada'},
 {'proveedores': ['Modas Paqui'],
  'nombre_producto': 'Camiseta de manga corta negra'},
 {'proveedores': ['Proveedor Este'], 'nombre_producto': 'Bufanda de seda'},
 {'proveedores': ['Logí

In [7]:
# 3. Listado de todos los productos diferentes comprados por un cliente
pipeline = [
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": "$cliente.nombre",
            "productos": { "$addToSet": "$productos.nombre" }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "productos": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_cliente = list(Compra.aggregate(pipeline, raw=True))

# Muestra los resultados
productos_cliente    

[{'productos': ['Bufanda de seda',
   'Camiseta de manga corta blanca',
   'Chaqueta de cuero',
   'Calcetines de algodón',
   'Falda plisada',
   'Zapatos deportivos',
   'Camiseta de manga corta roja',
   'Sombrero de paja',
   'Zapatos de vestir',
   'Traje de baño',
   'Camiseta de manga corta azul'],
  'nombre_cliente': 'Carlos López'},
 {'productos': ['Traje de baño',
   'Zapatos deportivos',
   'Bufanda de seda',
   'Jersey de lana',
   'Calcetines de algodón'],
  'nombre_cliente': 'Ana Sánchez'},
 {'productos': ['Polo de manga corta verde',
   'Cinturón de cuero',
   'Jersey de lana',
   'Camiseta de manga corta roja',
   'Sombrero de paja',
   'Falda plisada',
   'Camiseta de manga corta negra',
   'Camiseta de manga corta blanca',
   'Zapatos de vestir',
   'Bufanda de seda',
   'Zapatos deportivos',
   'Guantes de invierno',
   'Pantalón vaquero',
   'Camisa de manga larga',
   'Chaqueta de cuero'],
  'nombre_cliente': 'María Fernández'},
 {'productos': ['Camiseta de manga c

In [8]:
# 4. Listado de productos vendidos por “Modas Paqui” cuyo nombre contenga “manga corta”
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$match": {
            "proveedores.nombre": "Modas Paqui",
            "nombre": { "$regex": "manga corta", "$options": "i" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$nombre",
            "proveedor": "$proveedores.nombre"
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_manga_corta = list(Producto.aggregate(pipeline, raw=True))

productos_manga_corta

[{'nombre_producto': 'Camiseta de manga corta azul',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta blanca',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta negra',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta roja',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Polo de manga corta verde', 'proveedor': 'Modas Paqui'}]

In [9]:
# 5. Calcular el peso y volumen total de los productos comprados por un cliente un día determinado
pipeline = [
    {
        "$match": {
            "cliente.nombre": "Beatriz Gómez",
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": None,
            "peso_total_kg": { "$sum": "$productos.peso" },
            "volumen_total_metros": {
                "$sum": {
                    "$multiply": [
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.ancho" }, 100] },
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.alto" }, 100] },
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.profundidad" }, 100] }
                    ]
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "peso_total_kg": 1,
            "volumen_total_metros": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
peso_volumen_cliente = list(Compra.aggregate(pipeline, raw=True))

peso_volumen_cliente

[{'peso_total_kg': 106.59, 'volumen_total_metros': 0.27638}]

In [11]:
# 6. Calcular el número medio de envíos por mes y almacén
pipeline = [
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$group": {
            "_id": {
                "almacen": "$productos.proveedores.direcciones_almacenes.calle",
                "mes": { "$month": "$fecha_compra" },
                "año": { "$year": "$fecha_compra" }
            },
            "total_envios": { "$sum": 1 }
        }
    },
    {
        "$group": {
            "_id": "$_id.almacen",
            "promedio_envios_mensuales": { "$avg": "$total_envios" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "almacen": "$_id",
            "promedio_envios_mensuales": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
promedio_envios = list(Compra.aggregate(pipeline, raw=True))

promedio_envios

[{'promedio_envios_mensuales': 1.1666666666666667,
  'almacen': 'Calle de Alcalá'},
 {'promedio_envios_mensuales': 2.0, 'almacen': 'Carrer de la Pau'},
 {'promedio_envios_mensuales': 2.7142857142857144,
  'almacen': 'Calle de la Moda'},
 {'promedio_envios_mensuales': 4.0, 'almacen': 'Rua Augusta'},
 {'promedio_envios_mensuales': 1.1428571428571428,
  'almacen': 'Avenida de Andalucía'}]

In [12]:
# 7. Listado con los tres proveedores con más volumen de facturación
pipeline = [
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    {
        "$group": {
            "_id": "$productos.proveedores.nombre",
            "volumen_facturacion": { "$sum": "$precio_compra" }
        }
    },
    { "$sort": { "volumen_facturacion": -1 } },
    { "$limit": 3 },
    {
        "$project": {
            "_id": 0,
            "proveedor": "$_id",
            "volumen_facturacion": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
top_proveedores = list(Compra.aggregate(pipeline, raw=True))

top_proveedores

[{'volumen_facturacion': 14796.98, 'proveedor': None},
 {'volumen_facturacion': 7822.58, 'proveedor': 'Modas Paqui'},
 {'volumen_facturacion': 7280.96, 'proveedor': 'Proveedor Este'}]

In [15]:
# 8. Listado de almacenes cerca de unas coordenadas determinadas (100km de distancia máxima)
pipeline = [
    {
        "$geoNear": {
            "near": { "type": "Point", "coordinates": [-9.537398, 38.710141] },
            "distanceField": "distancia",
            "maxDistance": 100 * 1000,
            "spherical": True,
            "query": { "direcciones_almacenes.location": { "$exists": True } }
        }
    },
    { "$unwind": "$direcciones_almacenes" },
    { "$sort": { "distancia": 1 } },
    {
        "$project": {
            "_id": 0,
            "nombre_proveedor": "$nombre",
            "direccion_almacen": "$direcciones_almacenes.calle",
            "ciudad": "$direcciones_almacenes.ciudad",
            "distancia": 1,
            "location": "$direcciones_almacenes.location"
        }
    }
]

# Ejecuta el pipeline con raw=True
almacenes_cercanos = list(Proveedor.aggregate(pipeline, raw=True))

almacenes_cercanos

[{'distancia': 34745.68888511416,
  'nombre_proveedor': 'Proveedor Este',
  'direccion_almacen': 'Rua Augusta',
  'ciudad': 'Lisboa',
  'location': {'type': 'Point', 'coordinates': [-9.137398, 38.710141]}}]

In [4]:
# 9. Listado de compras dentro de un polígono cuyos vértices están definidos
pipeline = [
    {
        "$match": {
            "direccion_envio.location": {
                "$geoWithin": {
                    "$geometry": {
                        "type": "Polygon",
                        "coordinates": [[
                            [-3.8880, 40.5687],
                            [-3.5499, 40.5687],
                            [-3.5499, 40.3120],
                            [-3.8880, 40.3120],
                            [-3.8880, 40.5687]
                        ]]
                    }
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "compra_id": "$_id",
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "direccion_envio": "$direccion_envio",
            "location": "$direccion_envio.location"
        }
    }
]

# Ejecuta el pipeline con raw=True
compras_poligono = list(Compra.aggregate(pipeline, raw=True))

compras_poligono

[]

In [16]:
# 10. Guardar el listado de compras en una nueva colección
pipeline = [
    {
        "$match": {
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$match": {
            "productos.proveedores.direcciones_almacenes.calle": "Avenida de Andalucía",
            "productos.proveedores.direcciones_almacenes.ciudad": "Sevilla",
            "productos.proveedores.direcciones_almacenes.location.coordinates": [-5.963709, 37.387533]
        }
    },
    {
        "$project": {
            "_id": 1,
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "precio_compra": 1,
            "fecha_compra": 1,
            "direccion_envio": 1,
            "almacen": {
                "direccion": "$productos.proveedores.direcciones_almacenes.calle",
                "ciudad": "$productos.proveedores.direcciones_almacenes.ciudad",
                "location": "$productos.proveedores.direcciones_almacenes.location"
            }
        }
    },
    {
        "$out": "compras_a_enviar"
    }
]

Compra.aggregate(pipeline)

compras_a_enviar = list(Compra.db.database["compras_a_enviar"].find())
compras_a_enviar

[{'_id': ObjectId('671e87d36018dfcfed045212'),
  'precio_compra': 158.92000000000002,
  'fecha_compra': datetime.datetime(2024, 4, 11, 0, 0),
  'direccion_envio': None,
  'cliente': 'Beatriz Gómez',
  'productos': 'Gafas de sol',
  'almacen': {'direccion': 'Avenida de Andalucía',
   'ciudad': 'Sevilla',
   'location': {'type': 'Point', 'coordinates': [-5.963709, 37.387533]}}},
 {'_id': ObjectId('671e87d36018dfcfed045213'),
  'precio_compra': 471.22,
  'fecha_compra': datetime.datetime(2024, 4, 11, 0, 0),
  'direccion_envio': None,
  'cliente': 'Beatriz Gómez',
  'productos': 'Traje de baño',
  'almacen': {'direccion': 'Avenida de Andalucía',
   'ciudad': 'Sevilla',
   'location': {'type': 'Point', 'coordinates': [-5.963709, 37.387533]}}}]